<a href="https://colab.research.google.com/github/muhuuh/rl_educ/blob/main/german_auudio_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ck0aj24b/unsloth_5393b2bf59c44b67ad06a0ea275f8dcf
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ck0aj24b/unsloth_5393b2bf59c44b67ad06a0ea275f8dcf
  Resolved https://github.com/unslothai/unsloth.git to commit 4cb7229ac1c346e143524b6f9a6ad544259364d6
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
     

In [37]:
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"

In [38]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name="LiquidAI/LFM2.5-1.2B-Instruct",  # native model for fine-tuning
    model_name = 'unsloth/Phi-4-mini-instruct-unsloth-bnb-4bit',
    max_seq_length=2048,  # Matches model context; adjust based on your dataset
    dtype=None,  # Auto-detect (bfloat16 if available for better stability)
    load_in_4bit=True  # Enables 4-bit quantization for efficient fine-tuning
)

==((====))==  Unsloth 2026.1.4: Fast Phi3 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [39]:
from datasets import load_dataset  # Import this for loading JSONL
import json

# Load the JSONL directly as a Hugging Face Dataset (handles large files efficiently)
ds = load_dataset("json", data_files="./sample_data/german_dictation_cleanup_dataset.jsonl")["train"]

def to_text(ex):
    resp = ex["response"]
    if not isinstance(resp, str):
        resp = json.dumps(resp, ensure_ascii=False)
    msgs = [
        {"role": "user", "content": ex["prompt"]},
        {"role": "assistant", "content": resp},
    ]
    return {
        "text": tokenizer.apply_chat_template(
            msgs, tokenize=False, add_generation_prompt=False
        )
    }

# Apply mapping: Format into chat template and keep only "text" for SFTTrainer
dataset = ds.map(to_text, remove_columns=ds.column_names)

In [40]:
print(dataset[3]["text"])

<|user|>wir treffen uns um 14: 00 am Eingang und gehen dann zusammen rein.<|end|><|assistant|>Wir treffen uns um 14:00 am Eingang und gehen dann zusammen rein.<|end|>


In [41]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
import torch

# Apply LoRA adapters explicitly for quantized model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Low rank for efficiency
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Memory-saving during backprop
    random_state=3407,  # For reproducibility (like seed)
    max_seq_length=2048
)

# Now proceed with SFTTrainer as before
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=SFTConfig(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=200,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs"
    )
)

trainer.train()

Unsloth: Making `model.base_model.model.model` require gradients


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,200 | Num Epochs = 3 | Total steps = 200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 8,912,896 of 3,844,934,656 (0.23% trained)


Step,Training Loss
1,3.484900
2,3.554000
3,3.613400
4,3.668900
5,3.443700
6,3.295100
7,3.300600
8,3.099600
9,2.877400
10,2.861900


Unsloth: Will smartly offload gradients to save VRAM!


KeyboardInterrupt: 

In [ ]:
model.save_pretrained_gguf("german_audio_correction2_gguf", tokenizer, quantization_method="q4_k_m", maximum_memory_usage = 0.7)

In [ ]:
from unsloth import FastLanguageModel

# Enable faster inference mode (after training)
model = FastLanguageModel.for_inference(model)

# Example test prompt (from your dataset/screenshot; user role for correction)
messages = [
    {"role": "user", "content": "ich habe das Ticket erstellt; die die also Nummer ahm genau ist SUP-5555."}
]

# Apply chat template and tokenize
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Add assistant start for generation
    return_tensors="pt"
).to("cuda")  # Use GPU if available

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,  # Limit output length
    temperature=0.7,  # Balanced for corrections
    top_p=0.95,
    do_sample=True
)

# Decode and print
response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print(response)

# Test another: "Ich glaube dass wir dass bis Sonntag schaffen wenn wir heute anfangen"
messages2 = [{"role": "user", "content": "Ich glaube dass wir dass bis Sonntag schaffen wenn wir heute anfangen"}]
inputs2 = tokenizer.apply_chat_template(messages2, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs2 = model.generate(input_ids=inputs2, max_new_tokens=128, temperature=0.7, top_p=0.95, do_sample=True)
print(tokenizer.decode(outputs2[0][len(inputs2[0]):], skip_special_tokens=True))